### https://www.census.gov/data/developers/data-sets.html

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

handle = dataiku.Folder("IO")
path=handle.get_info().get('accessInfo').get('root')

### check variables

In [3]:
import requests
from urllib.request import urlopen
import json

In [5]:
survey=[['annual business survey','https://api.census.gov/data/2018/abscs/'],
        ['American Community Survey Migration Flows','https://api.census.gov/data/2018/acs/flows/'],
        ['Annual Public Sector Statistics','https://api.census.gov/data/timeseries/govs/'],
        ['Annual Survey of Entrepreneurs','https://api.census.gov/data/2016/ase/csa/'],
        ['Annual Survey of Manufactures','https://api.census.gov/data/timeseries/asm/area2017/'],
        ['Business Dynamics Statistics','https://api.census.gov/data/timeseries/bds/'],
        ['ZIP Codes Business Patterns','https://api.census.gov/data/2018/zbp/'],
        ['Decennial Census','https://api.census.gov/data/2010/dec/sf1/'],
        ['Economic Census','https://api.census.gov/data/2017/ecnbasic/'],
        ['Health Insurance Statistics','https://api.census.gov/data/timeseries/healthins/sahie/'],
        ['International Data Base','https://api.census.gov/data/timeseries/idb/5year/'],
        ['International Trade','https://api.census.gov/data/timeseries/intltrade/exports/hs/'],
        ['Language Statistics','https://api.census.gov/data/2013/language/'],
        ['Planning Database','https://api.census.gov/data/2020/pdb/statecounty/'],
        ['Population Estimates and Projections','https://api.census.gov/data/2017/popproj/pop/'],
        ['Post-Secondary Employment Outcomes','https://api.census.gov/data/timeseries/pseo/earnings/'],
        ['Poverty Statistics','https://api.census.gov/data/timeseries/poverty/histpov2/'],
        ['Quarterly Workforce Indicators',''],
        ['Survey of Business Owners','https://api.census.gov/data/2012/sbo/variables.html']
       ]

In [6]:
def clean_names(x):
    p=x.find('<th>')
    if p>-1:
        x=x[(p+4):]
    p=x.find('</th>')
    if p>-1:
        x=x[:p]
    return(x)

def clean_tbl(x):
    if x.find('N/A')>-1:
        return('N/A')
    else:
        p=x.find('>')
        if p>-1:
            x=x[(p+1):]
        p=x.find('<')
        if p>-1:
            x=x[:p]
        return(x)

In [7]:
variables=pd.DataFrame()

for s in survey:
    print(s[0])
    if s[1]!='':
        r=requests.get(s[1]+'variables.html')
        r=r.text
        
        p=r.find('<table>')
        p=r.find('<table>')
        r=r[p:]
        p1=r.find('<thead>')
        r=r[p1:]
        r=r.split('\n<tr>\n<td>')
        
        cols=[clean_names(x) for x in r[0].split('</th><th>')]
        y=[[clean_tbl(y) for y in x.split('</td><td>')] for x in r[1:]]
        
        v=pd.DataFrame(y,columns=cols)
        v['purpose']=s[0]
        
        variables=pd.concat([variables,v])

annual business survey
American Community Survey Migration Flows
Annual Public Sector Statistics
Annual Survey of Entrepreneurs
Annual Survey of Manufactures
Business Dynamics Statistics
ZIP Codes Business Patterns
Decennial Census
Economic Census
Health Insurance Statistics
International Data Base
International Trade
Language Statistics
Planning Database
Population Estimates and Projections
Post-Secondary Employment Outcomes
Poverty Statistics
Quarterly Workforce Indicators
Survey of Business Owners


/hadoopfs/fs1/dataiku/data_dir/code-envs/python/python36_code-env/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [8]:
variables.to_csv(path+'/00_non_acs_variables.csv',index=False)

### get data

In [80]:
from urllib.request import urlopen

url='https://api.census.gov/data/2019/acs/acs5'
columns='NAME,group(B01001)'
region='zip%20code%20tabulation%20area:*' #'county' #'state' # 'us:1'
key=''

link = """https://api.census.gov/data/timeseries/intltrade/exports/enduse?get=DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR&YEAR=2016&MONTH=06&key="""+key
link=url+'?get='+columns+'&for='+region+'&key='+key
with urlopen(link) as x:
     data = x.read().decode('utf-8')
        

In [82]:
p=data.find(']')
df=pd.read_json('['+data[(p+3):])
df.columns=data[2:(p)].replace('"','').split(',')

In [83]:
df.shape

(33120, 201)

In [84]:
df[:3]

,NAME,B01001_001E,B01001_001EA,B01001_001M,B01001_001MA,B01001_002E,B01001_002EA,B01001_002M,B01001_002MA,B01001_003E,B01001_003EA,B01001_003M,B01001_003MA,B01001_004E,B01001_004EA,B01001_004M,B01001_004MA,B01001_005E,B01001_005EA,B01001_005M,B01001_005MA,B01001_006E,B01001_006EA,B01001_006M,B01001_006MA,B01001_007E,B01001_007EA,B01001_007M,B01001_007MA,B01001_008E,B01001_008EA,B01001_008M,B01001_008MA,B01001_009E,B01001_009EA,B01001_009M,B01001_009MA,B01001_010E,B01001_010EA,B01001_010M,B01001_010MA,B01001_011E,B01001_011EA,B01001_011M,B01001_011MA,B01001_012E,B01001_012EA,B01001_012M,B01001_012MA,B01001_013E,B01001_013EA,B01001_013M,B01001_013MA,B01001_014E,B01001_014EA,B01001_014M,B01001_014MA,B01001_015E,B01001_015EA,B01001_015M,B01001_015MA,B01001_016E,B01001_016EA,B01001_016M,B01001_016MA,B01001_017E,B01001_017EA,B01001_017M,B01001_017MA,B01001_018E,B01001_018EA,B01001_018M,B01001_018MA,B01001_019E,B01001_019EA,B01001_019M,B01001_019MA,B01001_020E,B01001_020EA,B01001_020M,B01001_020MA,B01001_021E,B01001_021EA,B01001_021M,B01001_021MA,B01001_022E,B01001_022EA,B01001_022M,B01001_022MA,B01001_023E,B01001_023EA,B01001_023M,B01001_023MA,B01001_024E,B01001_024EA,B01001_024M,B01001_024MA,B01001_025E,B01001_025EA,B01001_025M,B01001_025MA,B01001_026E,B01001_026EA,B01001_026M,B01001_026MA,B01001_027E,B01001_027EA,B01001_027M,B01001_027MA,B01001_028E,B01001_028EA,B01001_028M,B01001_028MA,B01001_029E,B01001_029EA,B01001_029M,B01001_029MA,B01001_030E,B01001_030EA,B01001_030M,B01001_030MA,B01001_031E,B01001_031EA,B01001_031M,B01001_031MA,B01001_032E,B01001_032EA,B01001_032M,B01001_032MA,B01001_033E,B01001_033EA,B01001_033M,B01001_033MA,B01001_034E,B01001_034EA,B01001_034M,B01001_034MA,B01001_035E,B01001_035EA,B01001_035M,B01001_035MA,B01001_036E,B01001_036EA,B01001_036M,B01001_036MA,B01001_037E,B01001_037EA,B01001_037M,B01001_037MA,B01001_038E,B01001_038EA,B01001_038M,B01001_038MA,B01001_039E,B01001_039EA,B01001_039M,B01001_039MA,B01001_040E,B01001_040EA,B01001_040M,B01001_040MA,B01001_041E,B01001_041EA,B01001_041M,B01001_041MA,B01001_042E,B01001_042EA,B01001_042M,B01001_042MA,B01001_043E,B01001_043EA,B01001_043M,B01001_043MA,B01001_044E,B01001_044EA,B01001_044M,B01001_044MA,B01001_045E,B01001_045EA,B01001_045M,B01001_045MA,B01001_046E,B01001_046EA,B01001_046M,B01001_046MA,B01001_047E,B01001_047EA,B01001_047M,B01001_047MA,B01001_048E,B01001_048EA,B01001_048M,B01001_048MA,B01001_049E,B01001_049EA,B01001_049M,B01001_049MA,GEO_ID,NAME,state,zip code tabulation area
0,ZCTA5 25245,600,NaN,266,None,282,NaN,147,None,0,NaN,12,None,0,NaN,12,NaN,20,NaN,31,NaN,0,NaN,12,None,0,NaN,12,None,14,NaN,25,NaN,0,NaN,12,NaN,29,NaN,46,NaN,0,NaN,12,None,0,NaN,12,None,20,NaN,31,NaN,38,NaN,55,NaN,0,NaN,12,None,33,NaN,49,None,45,NaN,47,NaN,0,NaN,12,NaN,0,NaN,12,NaN,9,NaN,15,NaN,9,NaN,15,NaN,0,NaN,12,NaN,52,NaN,41,NaN,13,NaN,21,NaN,0,NaN,12,NaN,318,NaN,152,None,0,NaN,12,None,0,NaN,12,NaN,0,NaN,12,NaN,18,NaN,28,None,0,NaN,12,None,20,NaN,36,NaN,0,NaN,12,NaN,0,NaN,12,NaN,0,NaN,12,None,20,NaN,32,None,40,NaN,60,NaN,0,NaN,12,NaN,20,NaN,31,None,34,NaN,41,None,20,NaN,24,NaN,0,NaN,12,NaN,0,NaN,12,NaN,53,NaN,45,NaN,18,NaN,29,NaN,35,NaN,40,NaN,0,NaN,12,NaN,40,NaN,47,NaN,0,NaN,12,NaN,8600000US25245,ZCTA5 25245,54,25245
1,ZCTA5 25268,964,NaN,282,None,473,NaN,164,None,54,NaN,46,None,19,NaN,30,NaN,0,NaN,12,NaN,0,NaN,12,None,0,NaN,12,None,0,NaN,12,NaN,0,NaN,12,NaN,33,NaN,32,NaN,0,NaN,12,None,40,NaN,43,None,8,NaN,14,NaN,18,NaN,21,NaN,0,NaN,12,None,55,NaN,38,None,82,NaN,92,NaN,7,NaN,11,NaN,10,NaN,17,NaN,52,NaN,60,NaN,29,NaN,46,NaN,24,NaN,26,NaN,32,NaN,36,NaN,0,NaN,12,NaN,10,NaN,15,NaN,491,NaN,146,None,45,NaN,33,None,42,NaN,52,NaN,0,NaN,12,NaN,0,NaN,12,None,0,NaN,12,None,0,NaN,12,NaN,0,NaN,12,NaN,26,NaN,33,NaN,22,NaN,33,None,35,NaN,40,None,0,NaN,12,NaN,20,NaN,33,NaN,9,NaN,15,None,32,NaN,34,None,36,NaN,35,NaN,34,NaN,52,NaN,9,NaN,15,NaN,0,NaN,12,NaN,25,NaN,27,NaN,69,NaN,68,NaN,60,NaN,60,NaN,27,NaN,27,NaN,0,NaN,12,NaN,8600000US25268,ZCTA5 25268,54,25268
2,ZCTA5 25286,1700,NaN,435,No